1  ¿Cuál es la mejor tarifa?

Trabajas como analista para el operador de telecomunicaciones Megaline. La empresa ofrece a sus clientes dos tarifas de prepago, Surf y Ultimate. El departamento comercial quiere saber cuál de las tarifas genera más ingresos para poder ajustar el presupuesto de publicidad.

Vas a realizar un análisis preliminar de las tarifas basado en una selección de clientes relativamente pequeña. Tendrás los datos de 500 clientes de Megaline: quiénes son los clientes, de dónde son, qué tarifa usan, así como la cantidad de llamadas que hicieron y los mensajes de texto que enviaron en 2018. Tu trabajo es analizar el comportamiento de los clientes y determinar qué tarifa de prepago genera más ingresos.

[Te proporcionamos algunos comentarios para orientarte mientras completas este proyecto. Pero debes asegurarte de eliminar todos los comentarios entre corchetes antes de entregar tu proyecto.]

[Antes de sumergirte en el análisis de datos, explica por tu propia cuenta el propósito del proyecto y las acciones que planeas realizar.]

[Ten en cuenta que estudiar, modificar y analizar datos es un proceso iterativo. Es normal volver a los pasos anteriores y corregirlos/ampliarlos para permitir nuevos pasos.]

1.1  Inicialización

In [1]:
# Cargar todas las librerías
import pandas as pd
import numpy as np
from scipy import stats as st
import math as mt
from math import factorial


1.2  Cargar datos

In [2]:
# Carga los archivos de datos en diferentes DataFrames
calls_df = pd.read_csv('megaline_calls.csv')
internet_df = pd.read_csv('megaline_internet.csv')
messages_df = pd.read_csv('megaline_messages.csv')
plans_df = pd.read_csv('megaline_plans.csv')
users_df = pd.read_csv('megaline_users.csv')


1.3  Preparar los datos

[Los datos para este proyecto se dividen en varias tablas. Explora cada una para tener una comprensión inicial de los datos. Si es necesario, haz las correcciones requeridas en cada tabla.]

1.4  Tarifas

In [3]:
# Imprime la información general/resumida sobre el DataFrame de las tarifas
print(plans_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   messages_included      2 non-null      int64  
 1   mb_per_month_included  2 non-null      int64  
 2   minutes_included       2 non-null      int64  
 3   usd_monthly_pay        2 non-null      int64  
 4   usd_per_gb             2 non-null      int64  
 5   usd_per_message        2 non-null      float64
 6   usd_per_minute         2 non-null      float64
 7   plan_name              2 non-null      object 
dtypes: float64(2), int64(5), object(1)
memory usage: 260.0+ bytes
None


In [4]:
# Imprime una muestra de los datos para las tarifas
print('Muestra de los datos para las tarifas\n')
print(plans_df.head())

Muestra de los datos para las tarifas

   messages_included  mb_per_month_included  minutes_included  \
0                 50                  15360               500   
1               1000                  30720              3000   

   usd_monthly_pay  usd_per_gb  usd_per_message  usd_per_minute plan_name  
0               20          10             0.03            0.03      surf  
1               70           7             0.01            0.01  ultimate  


[Describe lo que ves y observas en la información general y en la muestra de datos impresa para el precio de datos anterior. ¿Hay algún problema (tipos de datos no adecuados, datos ausentes, etc.) que pudieran necesitar investigación y cambios adicionales? ¿Cómo se puede arreglar?]

El Dataset contiene 8 columnas, al parecer todas tiene los tipos correctos de datos y los nombres son claros a excepcion de usd_monthly_pay, creo que seria mejor usd_montly_fee, ninguna columna tiene datos ausentes. solo es actualizar el nombre de 1 columna.

1.5  Corregir datos

[Corrige los problemas obvios con los datos basándote en las observaciones iniciales.]

In [5]:
# Cambio de nombre en columna para mejor claridad y descripcion
plans_df = plans_df.rename(columns = {'mb_per_month_included': 'mb_included',
                                      'usd_monthly_pay': 'usd_monthly_fee',
                                      'usd_per_gb': 'usd_per_gb_exceeded',
                                      'usd_per_message': 'usd_per_message_exceeded',
                                      'usd_per_minute': 'usd_per_minute_exceeded'}
                          )
print(plans_df.columns)

Index(['messages_included', 'mb_included', 'minutes_included',
       'usd_monthly_fee', 'usd_per_gb_exceeded', 'usd_per_message_exceeded',
       'usd_per_minute_exceeded', 'plan_name'],
      dtype='object')


1.6  Enriquecer los datos

[Agrega factores adicionales a los datos si crees que pudieran ser útiles.]

In [6]:
# Agrega factores adicionales a los datos para mayor profundidad en el analisis
plans_df['plan_type'] = plans_df['usd_monthly_fee'].apply(lambda x: 'Premium' if x > 20 else 'Basic')
plans_df['usd_per_minute_cost'] = plans_df['usd_monthly_fee'] / plans_df['minutes_included']
plans_df['usd_per_gb_cost'] = plans_df['usd_monthly_fee'] / (plans_df['mb_included'] / 1240)
plans_df['usd_per_sms_cost'] = plans_df['usd_monthly_fee'] / plans_df['messages_included']

print('\nCosto unitario por servicio incluido en cada plan\n')
print(plans_df)


Costo unitario por servicio incluido en cada plan

   messages_included  mb_included  minutes_included  usd_monthly_fee  \
0                 50        15360               500               20   
1               1000        30720              3000               70   

   usd_per_gb_exceeded  usd_per_message_exceeded  usd_per_minute_exceeded  \
0                   10                      0.03                     0.03   
1                    7                      0.01                     0.01   

  plan_name plan_type  usd_per_minute_cost  usd_per_gb_cost  usd_per_sms_cost  
0      surf     Basic             0.040000         1.614583              0.40  
1  ultimate   Premium             0.023333         2.825521              0.07  


1.7  Usuarios/as

In [7]:
# Imprime la información general/resumida sobre el DataFrame de usuarios
print('Informacion general del dataset Usuarios\n')
print(users_df.info())

Informacion general del dataset Usuarios

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     500 non-null    int64 
 1   first_name  500 non-null    object
 2   last_name   500 non-null    object
 3   age         500 non-null    int64 
 4   city        500 non-null    object
 5   reg_date    500 non-null    object
 6   plan        500 non-null    object
 7   churn_date  34 non-null     object
dtypes: int64(2), object(6)
memory usage: 31.4+ KB
None


In [8]:
# Imprime una muestra de datos para usuarios
print('Vista general de dataset Usuarios\n')
print(users_df.head())
print('\nMuestra de datos para dataset Usuarios\n')
print(users_df.sample(10))

Vista general de dataset Usuarios

   user_id first_name  last_name  age                                   city  \
0     1000   Anamaria      Bauer   45  Atlanta-Sandy Springs-Roswell, GA MSA   
1     1001     Mickey  Wilkerson   28        Seattle-Tacoma-Bellevue, WA MSA   
2     1002     Carlee    Hoffman   36   Las Vegas-Henderson-Paradise, NV MSA   
3     1003   Reynaldo    Jenkins   52                          Tulsa, OK MSA   
4     1004    Leonila   Thompson   40        Seattle-Tacoma-Bellevue, WA MSA   

     reg_date      plan churn_date  
0  2018-12-24  ultimate        NaN  
1  2018-08-13      surf        NaN  
2  2018-10-21      surf        NaN  
3  2018-01-28      surf        NaN  
4  2018-05-23      surf        NaN  

Muestra de datos para dataset Usuarios

     user_id  first_name last_name  age  \
431     1431    Napoleon      Burt   74   
78      1078     Earnest      Gray   47   
61      1061      Maryam  Valencia   38   
388     1388         Thu      Noel   75   
385   

[Describe lo que ves y observas en la información general y en la muestra de datos impresa para el precio de datos anterior. ¿Hay algún problema (tipos de datos no adecuados, datos ausentes, etc.) que pudieran necesitar investigación y cambios adicionales? ¿Cómo se puede arreglar?]

Dentro del dataset se pueden hacer algunas correcciones en los nombres para que sean mas claros, tipo de datos y rellenar los valores nulos. Tambien note que el nombre de la cuidad de residencia del usuario tiene multiples ciudades, entonces podriamos filtrar los datos por la primera cuidad solamente para hacer un analisis mas correcto en el futuro.

1.7.1  Corregir los datos

[Corrige los problemas obvios con los datos basándote en las observaciones iniciales.]

In [9]:
# Modificacion de nombre en columna
users_df = users_df.rename(columns={'reg_date': 'registration_date'})

# Modificacion de datatype en columnas
users_df['registration_date'] = pd.to_datetime(users_df['registration_date'])
users_df['churn_date'] = pd.to_datetime(users_df['churn_date'], errors='coerce')

# Verify changes on dataset
print(users_df.head())
print()
print(users_df.info())

   user_id first_name  last_name  age                                   city  \
0     1000   Anamaria      Bauer   45  Atlanta-Sandy Springs-Roswell, GA MSA   
1     1001     Mickey  Wilkerson   28        Seattle-Tacoma-Bellevue, WA MSA   
2     1002     Carlee    Hoffman   36   Las Vegas-Henderson-Paradise, NV MSA   
3     1003   Reynaldo    Jenkins   52                          Tulsa, OK MSA   
4     1004    Leonila   Thompson   40        Seattle-Tacoma-Bellevue, WA MSA   

  registration_date      plan churn_date  
0        2018-12-24  ultimate        NaT  
1        2018-08-13      surf        NaT  
2        2018-10-21      surf        NaT  
3        2018-01-28      surf        NaT  
4        2018-05-23      surf        NaT  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   user_id            500 non-null    int64 

1.7.2  Enriquecer los datos

[Agrega factores adicionales a los datos si crees que pudieran ser útiles.]

In [10]:
#Obtener la duracion de suscripcion por cliente
users_df['subscription_duration'] = (users_df['churn_date'] - users_df['registration_date']).dt.days
print('Duracion de suscripcion por cliente\n')
print(users_df['subscription_duration'])

# Verificar datos duplicados
duplicates = users_df[users_df.duplicated(subset=['user_id'], keep=False)]
print('usuarios_duplicados: ', duplicates)

Duracion de suscripcion por cliente

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
495   NaN
496   NaN
497   NaN
498   NaN
499   NaN
Name: subscription_duration, Length: 500, dtype: float64
usuarios_duplicados:  Empty DataFrame
Columns: [user_id, first_name, last_name, age, city, registration_date, plan, churn_date, subscription_duration]
Index: []


1.8  Llamadas

In [11]:
# Imprime la información general/resumida sobre el DataFrame de las llamadas
print(calls_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137735 entries, 0 to 137734
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   id         137735 non-null  object 
 1   user_id    137735 non-null  int64  
 2   call_date  137735 non-null  object 
 3   duration   137735 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 4.2+ MB
None


In [12]:
# Imprime una muestra de datos para las llamadas
print(calls_df.sample(10))

              id  user_id   call_date  duration
124523   1440_39     1440  2018-10-06      9.14
36987   1140_271     1140  2018-08-05     17.45
136212   1494_54     1494  2018-11-30      4.82
35590    1133_81     1133  2018-10-11      9.68
134354  1488_541     1488  2018-06-20     12.72
127180    1455_2     1455  2018-11-27      5.00
50138   1181_450     1181  2018-07-22      7.14
65787   1237_394     1237  2018-12-01     15.59
62812   1226_211     1226  2018-11-20     11.61
30984   1116_118     1116  2018-08-17      0.00


[Describe lo que ves y observas en la información general y en la muestra de datos impresa para el precio de datos anterior. ¿Hay algún problema (tipos de datos no adecuados, datos ausentes, etc.) que pudieran necesitar investigación y cambios adicionales? ¿Cómo se puede arreglar?]

Creo que en este dataset podriamos hacer algunos cambios en el nobmre de las columnas 'id' and 'duration' para que sean mas homogeneos los nombres, tambien cambiar el datatype de 'call_date' por datetime64,

1.8.1  Corregir los datos

[Corrige los problemas obvios con los datos basándote en las observaciones iniciales.]

In [13]:
# Modify calls_df columns name
calls_df = calls_df.rename(columns={'id': 'caller_id', 'duration': 'call_duration'})

# Modify datatype for column 'call_date'
calls_df['call_date'] = pd.to_datetime(calls_df['call_date'])

# Verify changes on dataset
print(calls_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137735 entries, 0 to 137734
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   caller_id      137735 non-null  object        
 1   user_id        137735 non-null  int64         
 2   call_date      137735 non-null  datetime64[ns]
 3   call_duration  137735 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 4.2+ MB
None


1.8.2  Enriquecer los datos

[Agrega factores adicionales a los datos si crees que pudieran ser útiles.]

In [14]:
# Analizar llamadas por usuario
calls_per_user = calls_df.groupby('user_id')['call_date'].count().reset_index()
print('Llamadas por usuario\n', calls_per_user)

# Cual es el maximo tiempo en duracion de llamadas
max_call_duration = calls_df['call_duration'].max()
print(f'\nDuracion maxima de una llamda: {max_call_duration} minutos\n')

# lapso de tiempo entre llamadas
# Ordenar por usuario y fecha de llamada
calls_df = calls_df.sort_values(by=['user_id', 'call_date'])
# calcular tiempo entre llamadas
calls_df['time_between_calls'] = calls_df.groupby('user_id')['call_date'].diff()
# Imprimir resultado
print('\nLapso de tiempo entre llamadas por usuario\n')
print(calls_df[['user_id', 'call_date', 'time_between_calls']])


Llamadas por usuario
      user_id  call_date
0       1000         16
1       1001        261
2       1002        113
3       1003        149
4       1004        370
..       ...        ...
476     1495        253
477     1496        195
478     1497         54
479     1498        451
480     1499        204

[481 rows x 2 columns]

Duracion maxima de una llamda: 37.6 minutos


Lapso de tiempo entre llamadas por usuario

        user_id  call_date time_between_calls
9          1000 2018-12-26                NaT
14         1000 2018-12-26             0 days
0          1000 2018-12-27             1 days
1          1000 2018-12-27             0 days
2          1000 2018-12-27             0 days
...         ...        ...                ...
137708     1499 2018-12-29             1 days
137734     1499 2018-12-29             0 days
137638     1499 2018-12-30             1 days
137587     1499 2018-12-31             1 days
137709     1499 2018-12-31             0 days

[137735 rows x 3 colum

1.9  Mensajes

In [15]:
# Imprime la información general/resumida sobre el DataFrame de los mensajes
print(messages_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76051 entries, 0 to 76050
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            76051 non-null  object
 1   user_id       76051 non-null  int64 
 2   message_date  76051 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.7+ MB
None


In [16]:
# Imprime una muestra de datos para los mensajes
print(messages_df.sample(10))


              id  user_id message_date
26218   1159_221     1159   2018-11-07
23602    1144_63     1144   2018-09-19
57038   1362_106     1362   2018-11-14
50993   1331_168     1331   2018-11-03
23487   1142_208     1142   2018-08-26
62779    1396_63     1396   2018-08-18
37187   1247_351     1247   2018-07-30
21588  1132_1002     1132   2018-09-15
33207    1209_16     1209   2018-10-21
42967   1281_253     1281   2018-08-29


[Describe lo que ves y observas en la información general y en la muestra de datos impresa para el precio de datos anterior. ¿Hay algún problema (tipos de datos no adecuados, datos ausentes, etc.) que pudieran necesitar investigación y cambios adicionales? ¿Cómo se puede arreglar?]

1.9.1  Corregir los datos

[Corrige los problemas obvios con los datos basándote en las observaciones iniciales.]

In [17]:
# Corregir el datatype de 'message_date'
messages_df['message_date'] = pd.to_datetime(messages_df['message_date'])

print(messages_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76051 entries, 0 to 76050
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            76051 non-null  object        
 1   user_id       76051 non-null  int64         
 2   message_date  76051 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 1.7+ MB
None
